# Creating VIIRS composites from the LEN repository

In [ ]:
import pandas as pd
import sys, os
import geopandas as gpd

from GOSTRocks.misc import tPrint

sys.path.append("../src")

import len_tools 
from len_tools import Catalog
from len_tools import VIIRS_cleaner

%load_ext autoreload
%autoreload 2

In [ ]:
aws_bucket = 'globalnightlight'
iso3 = 'AFG'
start_day = "2020-01-01"
end_day = "2020-02-28"
out_folder = "/home/wb411133/temp"

cat = Catalog()
adm0 = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
adm0 = adm0.loc[adm0.iso_a3 == iso3]

In [ ]:
# Generate catalog of images for country and time period
current_cat = cat.search_by_period(start_day, end_day)
current_cat = current_cat.search_by_intersect(adm0, calc_overlap=True)

In [ ]:
# Throw away scenes with coverage lower than 25%
sel_scenes = current_cat.scenes.loc[current_cat.scenes.overlap > 0.25]

In [ ]:
cleaner = VIIRS_cleaner(aws_bucket, sel_scenes, adm0.unary_union)
res = cleaner.clean_viirs_data()

In [ ]:
cleaner.write_output(out_folder, iso3)